In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import io
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import glob
import random

from scipy.optimize import curve_fit

from astropy.io import fits
from astropy.time import Time
from astropy import units as u
import scipy.integrate as integrate
import lightkurve

from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

import math
import datetime as dt
import lightkurve.io as l
from lightkurve.io.generic import read_generic_lightcurve

In [2]:
def clean_text(str):
    s = result = ''.join([i for i in archivo[20:] if not i.isdigit()])
    t = s.replace('_curves.txt', '')
    u = t.replace('+', '')
    u = u.replace('-', '')
    if 'KsJs' in u:
        u = u.replace('KsJs', 'Ks Js')
    if u[0] == '_':
        u = u[1:]
    if 'AB' in u:
        u = u.replace('AB', '')
    return u

In [3]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def textcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    y = str(x.split('  ')[2])
    z = y.replace("'", "")
    return z

In [4]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def numcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    x.split('  ')
    if ' NaN' in x:
        y = str(x.split('  ')[1])
        z = y.replace("'", "")
        a = z.replace(" ", "")
        return a
    else:
        w = str(str1).split('\n')[0]
        x = str(w)
        y = str(x.split('  ')[2])
        z = y.replace("'", "")
        return z

In [5]:
#repository like the one we have online
general_data = pd.read_csv('/Users/kmanj/FITS_BD_Files/Repository_Data/Repository.csv', encoding ='latin1')

In [6]:
dir = '/Users/kmanj/Manjavacas Research/Metchev_2015/Metchev2015/Variables/'
dir_save = '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/'

In [7]:
dwarfs_files = os.listdir(dir)
dwarf_dat = [archive for archive in dwarfs_files if archive.endswith(".txt")]
dwarf_dat;

In [8]:
new_name_file = []
new_name = []

# Iterate over the elements of the first list.
for archivo in dwarf_dat:
    if archivo.startswith('2M'):
        # Extract the first 5 characters of the file name if 2MASS object
        first_5_characteres = 'J' + str(archivo[2:6])
    elif archivo.startswith('DENIS'):
        first_5_characteres = 'J' + str(archivo[5:9])
    elif archivo.startswith('SDSS'):
        first_5_characteres = 'J' + str(archivo[4:8])
    elif archivo.startswith('HN'):
        first_5_characteres = 'J2144'
    
    # Iterate over the elements of the second list.
    for valor in general_data['Name_file']:
        # Extract the first 5 characters of the reference value
        valor = str(valor)
        first_5_values = valor[:5]
        
        # Compare the first 5 characters of the two elements
        if first_5_characteres == first_5_values:
            # Modify the file name to the desired format
            nuevo_nombre = valor + ".dat"
            # Add the new name to the new list
            new_name.append(valor)
            new_name_file.append(nuevo_nombre)
            # Break the inner loop once a match is found.
            break

new_name_file;
new_name;

In [9]:
dwarfs_data = []
for i in range(len(new_name_file)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat[i]), header=None, sep='\t')
    dwarfs_data.append(data)

In [10]:
dwarfs_data;

In [11]:
ch_dict = {}
ch_dict['J0036+18'] = (0, 215)
ch_dict['J0050-33'] = (0, 358)
ch_dict['J0103+19'] = (0, 377)
ch_dict['J082521 '] = (0, 324)
ch_dict['J1126-50'] = (0, 378)
ch_dict['J1324+63'] = (0, 187) #Ch2
ch_dict['J1511+06'] = (0, 378)
ch_dict['J1516+30'] = (0, 365)
ch_dict['J1615+49'] = (0, 378)
ch_dict['J1632+19'] = (0, 215)
ch_dict['J1721+33'] = (0, 362)
ch_dict['J1753-65'] = (0, 378)
ch_dict['J1821+14'] = (0, 270)
ch_dict['J2148+40'] = (0, 362)
ch_dict['J2208+29'] = (0, 370)
ch_dict['J2228-43'] = (0, 377)
ch_dict['J1058-15'] = (0, 201)
ch_dict['J2144+14'] = (0, 362)
ch_dict['J0107+00'] = (0, 378)

In [12]:
Reference = 'Metchev et al 2015 ApJ 799 154'
Instrument = 'Infrared Array Camera'
Date_obs = 'Published January 27, 2015'
Band_obs_1 = 'Ch1' ##This needs to be figured out 
Band_obs_2 = 'Ch2'

In [13]:
Object = []
SpectralType = []
Jmag = []
Hmag = []
Kmag = []
dJmag = []
dHmag = []
dKmag = []
RA = []
Dec = []



for i in range(len(new_name_file)): 
    Object.append(textcleaner(general_data['Name'][general_data['Name_file'] == new_name[i]]))
    SpectralType.append(textcleaner(general_data['SpT '][general_data['Name_file'] == new_name[i]]))
    Jmag.append(textcleaner(general_data['J'][general_data['Name_file'] == new_name[i]]))
    dJmag.append(textcleaner(general_data['dJ'][general_data['Name_file'] == new_name[i]]))
    Hmag.append(numcleaner(general_data['H'][general_data['Name_file'] == new_name[i]]))
    dHmag.append(numcleaner(general_data['dH'][general_data['Name_file'] == new_name[i]]))
    Kmag.append(numcleaner(general_data['K'][general_data['Name_file'] == new_name[i]]))
    dKmag.append(numcleaner(general_data['dK'][general_data['Name_file'] == new_name[i]]))
    RA.append(textcleaner(general_data['RA'][general_data['Name_file'] == new_name[i]]))
    Dec.append(textcleaner(general_data['DEC'][general_data['Name_file'] == new_name[i]]))

In [17]:
name_ch1

['/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/LSPM J0036+1821_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASS J00501994-3322402_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASSI J0103320+193536_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASSI J0825196+211552_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASS J11263991-5003550_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASS J13243553+6358281_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/SDSS J151114.66+060742.9_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/SDSS J151643.01+305344.4_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASS J16154255+4953211_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASSW J1632291+190441_Ch1_Metc2015.fits',
 '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/V/2MASSI J1721039+334415_Ch1_Met

In [18]:
RA = [
    9.067376,
    12.583083,
    15.883458,
    126.331999,
    171.666,
    201.148083,
    227.81108,
    229.1791,
    243.92729,
    248.121291,
    260.26625,
    268.438249,
    275.367291,
    327.068041,
    332.05679,
    337.120376,
    164.699458,
    326.130541,
    16.968417
]

Dec = [
    18.352889,
    -33.377833,
    19.593361,
    21.264472,
    -50.065281,
    63.974486,
    6.128639,
    30.8957,
    49.889194,
    19.077972,
    33.737778,
    -65.998861,
    14.233611,
    40.0665,
    29.355972,
    -43.173944,
    -15.804778,
    14.7719398,
    0.698972
]

In [21]:
name_ch1 = []
name_ch2 = []

for i in range(len(dwarfs_data)):
    if new_name[i] == 'J1324+63':
        ch2 = dwarfs_data[i].iloc[0: ch_dict[new_name[i]][1]]
        ch1 = dwarfs_data[i].iloc[ch_dict[new_name[i]][1]:]
    else:
        ch1 = dwarfs_data[i].iloc[0: ch_dict[new_name[i]][1]]
        ch2 = dwarfs_data[i].iloc[ch_dict[new_name[i]][1]:]
        
    lc1 = l.LightCurve(time = list(ch1[0]), flux = list(ch1[4]) * u.dimensionless_unscaled)
    lc2 = l.LightCurve(time = list(ch2[0]), flux = list(ch2[4]) * u.dimensionless_unscaled)
    
    lc1.to_fits(path = dir_save + Object[i] + '_Ch1_Metc2015.fits')
    lc2.to_fits(path = dir_save + Object[i] + '_Ch2_Metc2015.fits')
    name_ch1.append(dir_save + Object[i] + '_Ch1_Metc2015.fits')
    name_ch2.append(dir_save + Object[i] + '_Ch2_Metc2015.fits')

In [22]:
for i in range(len(dwarf_dat)):
    with fits.open(name_ch1[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = 'Spitzer Space Telescope'
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs_1
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] + '+/-' + dJmag[i]
        f[0].header['HMAG'] = Hmag[i] + '+/-' + dHmag[i]
        f[0].header['KMAG'] = Kmag[i] + '+/-' + dKmag[i]
        f[1].header['TUNIT1'] = 'Hrs'

In [23]:
for i in range(len(dwarf_dat)):
    with fits.open(name_ch2[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = 'Spitzer Space Telescope'
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs_2
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] + '+/-' + dJmag[i]
        f[0].header['HMAG'] = Hmag[i] + '+/-' + dHmag[i]
        f[0].header['KMAG'] = Kmag[i] + '+/-' + dKmag[i]
        f[1].header['TUNIT1'] = 'Hrs'